### Onboarding Waitlist 

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

In [2]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')
network_waitlist_data = pd.read_excel('network_waitlist.xlsx', sheet_name='Sheet1')

print(len(registration_data))
print(len(onboarding_data))
print(len(network_waitlist_data))

471
306
182


### Prepare Data and Merge 

In [5]:
network_waitlist_data = network_waitlist_data[~network_waitlist_data["Safe address"].isnull()]
print(len(network_waitlist_data))

180


In [6]:
# Strip of leading and tailing white space
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()
network_waitlist_data["Safe address"] = network_waitlist_data["Safe address"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]]
network_waitlist_data["Safe address"] = [x.lower() for x in network_waitlist_data["Safe address"]]

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration

print(len(registration_data))

471


In [8]:
registration_data[registration_data["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate
459,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False


In [9]:
# Only keep unique values
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

Duplicate
False    306
Name: count, dtype: int64
Duplicate
False    447
Name: count, dtype: int64


In [12]:
# registration_data_01[registration_data_01["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]
onboarding_data_01[onboarding_data_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,deployment_date,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,nft_id,Duplicate
139,2023-10-02 18:49:25.000 UTC,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,18592.0,False


In [13]:
## Delete people who are already in the waitlist from the registration data

x = list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])
print(len(x))

# Drop Avado users in original list (now no problem as only a few have avado it does not iterfer with organic)
registration_data_01['waitlist_ind'] = np.where(registration_data_01["What is your HOPR safe address?"].isin(x), "YES", "No")
registration_data_02 = registration_data_01[registration_data_01['waitlist_ind'] == "No"]
print(len(registration_data_02))

90
357


In [14]:
registration_data_02[registration_data_02["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate,waitlist_ind
459,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False,No


In [15]:
waitlist = registration_data_02.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()]
print(len(waitlist_01))
display(waitlist_01.head())

357
166


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
38,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False
49,2023-09-09T15:09:31.289Z,2023-09-09 15:02:25.000 UTC,0x557eae5968432ca52244f16bd364308777b49c63,0xc3fcab887a3f9b07776a47a2fbe452934cecac13,0xcd13c24468d36d45ad353156b8956a9b0b598e8d1372...,70000.000000,False
63,2023-09-09T15:50:12.825Z,2023-09-09 15:36:55.000 UTC,0x27d559ef0a9ab0d00dedbe5e0d5c215ec475ee24,0x4e4c24299bece3aff08b013d10023eb011394199,0xab810a4bac3fda9f2f5f6fa51818cdf0a7f9ad264216...,0.000037,False
65,2023-09-09T16:00:12.038Z,2023-09-09 13:32:15.000 UTC,0x9680ca3188117a40d6ce7c940020c7ac3afd58a4,0x865a0bef05fa813aae59ef12d88ace8eebef49d0,0x68a9e3e3c36c1b811beda8976d72e711d3fd156b9533...,0.000000,False
128,2023-09-10T07:07:53.959Z,2023-09-10 06:28:05.000 UTC,0xbd6e77e2b40e1613047479b0c7c720041d4e3f8f,0x138969cbb6b1346151986493bf82084eb98a5c2a,0x24203d9be19d0b0c109721eeb313afd9817cc16a2c45...,64257.000000,False


In [16]:
waitlist_01[waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


### Delete non-elidgible nodes 

In [17]:
# Not enough stake
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

161
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [18]:
waitlist_02[waitlist_02["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


In [19]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

nr_nft
True     125
False     36
Name: count, dtype: int64


In [20]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy()
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


eligible
True     160
False      1
Name: count, dtype: int64


In [21]:
waitlist_04[waitlist_04["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [22]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

160


In [23]:
waitlist_05[waitlist_05["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [24]:
waitlist_05["node_address"] = waitlist_05["node_address"].str.strip()

In [25]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

156


In [26]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [33]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


### Sort Users 

In [34]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True]
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

121
35


In [35]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
display(nr_waitlist_01.head())

121


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-03T22:21:50.350Z,2023-09-04 09:11:05.000 UTC,0x3e93116537defa88f216035fd38ee791c7720658,0x8f75312247f34b8eaf225b3ca0b38a28980070f1,0x4575a7b86e61cfff3e688a2cfef61286b39de2403552...,75000.000000,True,True
1,2023-09-20T07:31:27.974Z,2023-09-04 10:53:55.000 UTC,0x555b3459f7b66f4a8d1b1b843f9d79b45fc184b7,0x97e196006f0cbe6613e7cfcc0e61fcc596fa2827,0x7c4ff98ac14dedb97c954a11b52aa6fe2b8ff1658cc9...,21950.808430,True,True
2,2023-09-26T13:26:37.695Z,2023-09-06 10:43:20.000 UTC,0x6f9b56d7e8d4efaf0b9364f52972a1984a76e68b,0xe8d96badfdd3d4495079daa048dae099b059107c,0x8e75e78085ca034404341cd41a823ce156b193f50623...,10453.749841,True,True
3,2023-09-20T19:05:31.949Z,2023-09-07 22:56:00.000 UTC,0x75290dd22be7084e2fa2ab500d13d1047974dd1d,0x8bf86b3fc6f56a9a35c693641da7bb25d4a7224f,0x91aee206bd4d9f55bd7b897f42ef89a3fa6623ebbac3...,80212.625978,True,True
4,2023-09-25T13:19:05.372Z,2023-09-09 12:11:40.000 UTC,0x18ef4009b1e49af8dbffdda61ee5c4849b8a4f7f,0xf1a745131dd9954fadda87a4240a8352586422c0,0xad9ddc56019d5519c3baef26e45e963a4270eec2645e...,350000.000000,True,True


In [36]:
nr_waitlist_01[nr_waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
113,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [39]:
print(len(nr_waitlist_01))

121


In [29]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
display(stake_waitlist_01.head())

35


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-23T21:26:43.338Z,2023-09-23 20:50:45.000 UTC,0x4be394bd65d37e8bb2a94c3d6851ea3c3ac721f2,0xe4d2136b174f301368d43c0042d16fa1121eb25b,0xfda6286572d2c701f0ae258f3c8527d6ac283661dda0...,92922.580611,False,True
1,2023-09-23T20:39:53.847Z,2023-09-23 17:31:35.000 UTC,0xe306e4eee9bb0adf59b4c3d13700a410e80ac9ba,0xed97aa9ec17a268cffd0c14e786630de9106d88c,0x95cf5a419860ab7b7a981308fc21a09883e9a24ba24d...,80000.000000,False,True
2,2023-09-26T17:46:59.079Z,2023-09-26 15:39:20.000 UTC,0x9a9fc872bc36a77a0120a5215ca6a6275204a716,0xe1e80862f087928a6c170b4e4788ff680664e63b,0xbea93538bb110bd322a2b7399e3a21dcc85570968277...,75000.000000,False,True
3,2023-09-30T14:57:51.522Z,2023-09-30 14:50:15.000 UTC,0x0d49c8379895b4d1b8f17f176057364d564607ed,0x75af1a1e85e948cefd6bfbb6040f8617c447a471,0x3372d54d33bfd19d4fad601a4c21600a1b8d5f377d69...,75000.000000,False,True
4,2023-09-17T12:42:50.991Z,2023-09-17 12:39:55.000 UTC,0xf27fa15051617296f38d281283fdcbf8882407fc,0x586c8dcb85cb0f8c7f59d8fe0770183e8b3c8cf8,0xa09aa70deee86b19179a6b5a690d771083dad3454d8b...,75000.000000,False,True


In [40]:
print(len(stake_waitlist_01))

35


In [37]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) and stake_index < len(stake_waitlist_01):
    nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
    stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]

    final_chunks.extend([nr_chunk, stake_chunk])

    nr_index += nr_chunk_size
    stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-03T22:21:50.350Z,2023-09-04 09:11:05.000 UTC,0x3e93116537defa88f216035fd38ee791c7720658,0x8f75312247f34b8eaf225b3ca0b38a28980070f1,0x4575a7b86e61cfff3e688a2cfef61286b39de2403552...,75000.000000,True,True
1,2023-09-20T07:31:27.974Z,2023-09-04 10:53:55.000 UTC,0x555b3459f7b66f4a8d1b1b843f9d79b45fc184b7,0x97e196006f0cbe6613e7cfcc0e61fcc596fa2827,0x7c4ff98ac14dedb97c954a11b52aa6fe2b8ff1658cc9...,21950.808430,True,True
2,2023-09-26T13:26:37.695Z,2023-09-06 10:43:20.000 UTC,0x6f9b56d7e8d4efaf0b9364f52972a1984a76e68b,0xe8d96badfdd3d4495079daa048dae099b059107c,0x8e75e78085ca034404341cd41a823ce156b193f50623...,10453.749841,True,True
3,2023-09-20T19:05:31.949Z,2023-09-07 22:56:00.000 UTC,0x75290dd22be7084e2fa2ab500d13d1047974dd1d,0x8bf86b3fc6f56a9a35c693641da7bb25d4a7224f,0x91aee206bd4d9f55bd7b897f42ef89a3fa6623ebbac3...,80212.625978,True,True
4,2023-09-25T13:19:05.372Z,2023-09-09 12:11:40.000 UTC,0x18ef4009b1e49af8dbffdda61ee5c4849b8a4f7f,0xf1a745131dd9954fadda87a4240a8352586422c0,0xad9ddc56019d5519c3baef26e45e963a4270eec2645e...,350000.000000,True,True


In [44]:
import pandas as pd

# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

# Loop for nr_waitlist_01
while nr_index < len(nr_waitlist_01):
    nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
    final_chunks.append(nr_chunk)
    nr_index += nr_chunk_size

# Loop for stake_waitlist_01
while stake_index < len(stake_waitlist_01):
    stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]
    final_chunks.append(stake_chunk)
    stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-03T22:21:50.350Z,2023-09-04 09:11:05.000 UTC,0x3e93116537defa88f216035fd38ee791c7720658,0x8f75312247f34b8eaf225b3ca0b38a28980070f1,0x4575a7b86e61cfff3e688a2cfef61286b39de2403552...,75000.000000,True,True
1,2023-09-20T07:31:27.974Z,2023-09-04 10:53:55.000 UTC,0x555b3459f7b66f4a8d1b1b843f9d79b45fc184b7,0x97e196006f0cbe6613e7cfcc0e61fcc596fa2827,0x7c4ff98ac14dedb97c954a11b52aa6fe2b8ff1658cc9...,21950.808430,True,True
2,2023-09-26T13:26:37.695Z,2023-09-06 10:43:20.000 UTC,0x6f9b56d7e8d4efaf0b9364f52972a1984a76e68b,0xe8d96badfdd3d4495079daa048dae099b059107c,0x8e75e78085ca034404341cd41a823ce156b193f50623...,10453.749841,True,True
3,2023-09-20T19:05:31.949Z,2023-09-07 22:56:00.000 UTC,0x75290dd22be7084e2fa2ab500d13d1047974dd1d,0x8bf86b3fc6f56a9a35c693641da7bb25d4a7224f,0x91aee206bd4d9f55bd7b897f42ef89a3fa6623ebbac3...,80212.625978,True,True
4,2023-09-25T13:19:05.372Z,2023-09-09 12:11:40.000 UTC,0x18ef4009b1e49af8dbffdda61ee5c4849b8a4f7f,0xf1a745131dd9954fadda87a4240a8352586422c0,0xad9ddc56019d5519c3baef26e45e963a4270eec2645e...,350000.000000,True,True


In [45]:
print(len(final_waitlist))

156


In [38]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible


In [31]:
### sanity checks

# check users who already got access are not in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])).value_counts())

# check that no duplicates are in the waitlist
print((final_waitlist.duplicated(subset=['safe_address'], keep='last')).value_counts())

safe_address
False    115
Name: count, dtype: int64
False    115
Name: count, dtype: int64


In [32]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible


### Save final waitlist 

In [79]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)